In [68]:
import pandas as pd
import numpy as np

In [4]:
housing_data =pd.read_csv('housing.csv')

- 결측치
- 범주형 변수

- 중간 소득과 집 가격이 상관관계가 높다
- 독립변수들이 상관관계가 높다 (다중공선성) population, household, total_room, total_bed 들의 상관관계가 높다. 
- 변수들이 치우침이 큼.. (종속변수)
- 변수들별로 단위가 다름 

# 결측치 처리 

- 변수를 삭제 (열을 삭제) -> 별로 권하지 않음 
- 샘플을 삭제 (행을 삭제)
- 대체(imputation) -> 가급적 대체하는 쪽으로 생각하기 

# 변수 처리 

- 상관관계 높은 변수 처리 (다중공선성)
- 범주형 변수를 가변수(독립변수를 0과1로 변환한 변수를 의미)로 변환 (dummy variable)
- 변수간의 scale 표준화 혹은 정규화 -> 분석목적에 맞게 신중하게 판단해야 함 

In [5]:
housing_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


# 결측치 처리

In [6]:
housing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


- 범주형 변수의 경우는, median 값이라는 게 없다(= 대체할 수 있는 값이 없다). 때문에,범주형 변수를 dummy 변수(가변수)로 처리


- 결측치(total_bedrooms)를 중위수를 이용하여 imputation

In [10]:
# 1. 범주형 변수 따로 처리 
housing_num = housing_data.drop('ocean_proximity', axis=1)

In [11]:
housing_num.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [12]:
# 2. 결측치를 중위수를 이용하여 imputation(대체)

In [13]:
from sklearn.impute import SimpleImputer

In [14]:
imputer=SimpleImputer(strategy='median') # (median은 객체)imputer 를 통해서 median 값을 만듦

In [15]:
# fit
imputer.fit(housing_num)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [16]:
imputer.statistics_

array([-1.1849e+02,  3.4260e+01,  2.9000e+01,  2.1270e+03,  4.3500e+02,
        1.1660e+03,  4.0900e+02,  3.5348e+00,  1.7970e+05])

In [18]:
# transform
x =imputer.transform(housing_num) 
# ndarray numpy(배열을) -> pandas (판다스 형식을 바꿈)

In [21]:
# numpy -> pandas
imputed_housing_num = pd.DataFrame(x,columns=housing_num.columns, index=list(housing_num.index.values))

In [22]:
# 3. 범주형 변수를 dummy 변수(가변수)로 처리 

In [45]:
housing_cate=pd.get_dummies(housing_data['ocean_proximity'],drop_first=True) 
# 해당되는 값에 1을 넣었음. 

In [2]:
# ',drop_first=True' 로 1H_OCEAN 삭제. 다른 변수들이 0이면 그게 1H_OCEAN 을 의미하는 것이기 때문

In [46]:
housing_cate

,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
20635,1,0,0,0
20636,1,0,0,0
20637,1,0,0,0
20638,1,0,0,0


In [28]:
# 4. 수치형 변수들이 단위가 다르다../ 변수들별로 단위가 다름
# 표준화 
from sklearn.preprocessing import StandardScaler

In [29]:
std_scaler = StandardScaler()

In [30]:
std_scaler.fit(imputed_housing_num)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [33]:
# imputation -> 표준화 
x_std=std_scaler.transform(imputed_housing_num) #numpy 

In [36]:
# numpy -> pandas
std_imputed_housing_num = pd.DataFrame(x_std,columns=housing_num.columns, index=list(imputed_housing_num.index.values))

In [37]:
std_imputed_housing_num.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-1.327835,1.052548,0.982143,-0.804819,-0.972476,-0.974429,-0.977033,2.344766,2.129631
1,-1.322844,1.043185,-0.607019,2.045890,1.357143,0.861439,1.669961,2.332238,1.314156
2,-1.332827,1.038503,1.856182,-0.535746,-0.827024,-0.820777,-0.843637,1.782699,1.258693
3,-1.337818,1.038503,1.856182,-0.624215,-0.719723,-0.766028,-0.733781,0.932968,1.165100
4,-1.337818,1.038503,1.856182,-0.462404,-0.612423,-0.759847,-0.629157,-0.012881,1.172900


In [38]:
# 새로운 변수를 생성 (독립변수간의 상관관게 줄이고 더 유의미한 변수를 생성)

In [39]:
# 가구당 방 수 
std_imputed_housing_num['rooms_per_household']=std_imputed_housing_num['total_rooms']/std_imputed_housing_num['households']
# 방 대비 침실수
std_imputed_housing_num['bedrooms_per_room']=std_imputed_housing_num['total_bedrooms']/std_imputed_housing_num['total_rooms']
# 가구당 인구수
std_imputed_housing_num['population_per_household']=std_imputed_housing_num['population']/std_imputed_housing_num['households']

In [42]:
corr_matrix=std_imputed_housing_num.corr()

In [43]:
corr_matrix['households']

longitude                   5.531009e-02
latitude                   -7.103543e-02
housing_median_age         -3.029160e-01
total_rooms                 9.184845e-01
total_bedrooms              9.743663e-01
population                  9.072223e-01
households                  1.000000e+00
median_income               1.303305e-02
median_house_value          6.584265e-02
rooms_per_household         1.048959e-17
bedrooms_per_room           5.721078e-03
population_per_household   -7.073700e-18
Name: households, dtype: float64

In [44]:
# 단순 예측을 하려면 다중공선성이 문제가 안 될 수도 있다. 
# 하지만 회귀분석을 할 경우, 추 검정을 하게 되면 회귀계수의 신뢰도가 떨어지게 된다. 
# 해석이나 진단 측면에서 볼 때 문제가 될 수 있다. 

## merge

In [48]:
final_data = pd.merge(std_imputed_housing_num,housing_cate, left_index=True, right_index=True)
# 왼쪽 데이터와 오른쪽 데이터 인덱스를 기준으로 한다. 

In [51]:
final_data.head(20)
# total_rooms, total_bedrooms, population -> 새로 생성된 변수와 의미가 중복되기 때문에 분석에서 제거
# response variable(반응 변수): median_house_value
final_data2=final_data.drop(['total_rooms','total_bedrooms','population'],axis=1)

# Lecture_6

# Training

## traing vs testing

In [53]:
y = final_data2['median_house_value'] # 종속변수
x = final_data2.drop('median_house_value',axis=1) # 입력변수

In [52]:
## 데이터 쪼개기
from sklearn.model_selection import train_test_split

In [55]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

In [56]:
# training Linear_regression

In [57]:
from sklearn.linear_model import LinearRegression

In [58]:
lin_reg = LinearRegression() #lin_reg 이라는 객체로 저장 

In [59]:
lin_reg.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [64]:
# testing
predicted_value =lin_reg.predict(x_test)

In [63]:
# RMSE 
from sklearn.metrics import mean_squared_error

In [66]:
lin_mse = mean_squared_error(y_test,predicted_value)

In [69]:
np.sqrt(lin_mse)

0.6302117307510327

In [70]:
# 표준화 한 것을 역으로 값 형태로 바꿔줘야 함. 
# predicted_value*std + mean

In [71]:
housing_data['median_house_value'].std()

115395.6158744132

In [72]:
housing_data['median_house_value'].mean()

206855.81690891474

In [73]:
predicted_value*housing_data['median_house_value'].std()+housing_data['median_house_value'].mean()

array([ 87420.1090201 , 220083.91297473, 128655.32690078, ...,
       271489.97769698,  96167.86100126, 145565.02967209])

In [74]:
# decision tree -> 각자 해보기 

In [ ]:
# 보통은 전체 데이터의 70 퍼센트를 트레이닝 데이터, 30 퍼센트를 테스트 데이터로 나누면서
# 정확성을 검토하기는 하지만 이렇게 하기 위해서는 기본적으로 데이터의 양이 많아야 한다. 
# 데이터의 양이 충분히 많지 않을 경우 : k-fold (k개로 접는다) cross-validation 사용

## K-fold cross-validation

In [75]:
from sklearn.model_selection import cross_val_score

In [77]:
score=cross_val_score(lin_reg,x,y,scoring='neg_mean_squared_error',cv=10) # negative mean squared error k-fold = 10

In [80]:
np.sqrt(-score)

array([0.77249757, 0.53447632, 0.81896338, 0.58632826, 0.75019321,
       0.62091285, 0.46504781, 0.81068434, 0.69092644, 0.47823183])

In [81]:
# parameter 튜닝
# training 데이터를 train set, validation set 으로 나눠
# grid search로 하이퍼 파라미터 값을 정해줘.. 그리고 그에 해당하는 mean squared 값을 구해준다
# 이중에서 mean squared가 가장 작은 것을 골라 (비용함수, 목적함수, 손실함수)
# (그리트 서치 과정) 하이퍼 파라미터 튜닝을 할 때 비용함수(목적 함수)를 계산하여 가장 작은 값을 고른다. 
# 비용 함수는 주로 mse, rmse 값이 될 수 있다. 

# 비용 함수가 가장 작은 하이퍼 파라미터 값을 선택, 이 과정을 grid search에서 한다. 

# 시간이 오래걸린다. search space(값이 너무 많은 경우) 가 굉장히 넓을 경우엔 언제 다 해주냐.. 굉장히 오래걸림

# 그래서 search space 중에서 값을 random 하게 고르고 진행 --> random search

In [82]:
# 모델을 받고
# 스캔해서
# 전처리
# 기본적 시각화 -> 분석
# 트레이닝 
# 검정